# Create pyCIAM Storm Costs Lookup Table

Calculating the storm costs in a CIAM model involves a numerical integration over both elevation and the quantiles of storm surge at each segment-ADM1 location. This is too computationally intensive to run for all seg-ADMs for each year for all SLR trajectories, especially when using pyCIAM to run a Monte Carlo analysis across tens of thousands of SLR trajectories. Instead, we build a lookup table indexed by seg-ADM, LSLR, adaptation type (retreat vs. protect), cost type (mortality vs. capital loss), and `rhdiff` (the difference between the retreat/protect height and lslr). This is similar to how it is treated in the original CIAM model except that:

1. We use a lookup table rather than a parameterized exponential function of `rhdiff` and `lslr`
2. We account for elevational heterogeneity in population and capital when evaluating our costs in retreat scenarios. The original CIAM included `lslr` in their exponential function only for the protect adaptation type, while for `noAdaptation` and `retreat`, the function was only of `rhdiff`.

## Setup

In [1]:
import sys

sys.path.append("../")

In [2]:
import distributed as dd
import pandas as pd
from pyCIAM.surge import damage_funcs
from pyCIAM.surge.lookup import create_surge_lookup
from shared import (
    PATH_PARAMS,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_AR5_QUANTILES,
    PATH_SLR_AR6,
    PATH_SLR_SWEET,
    PATHS_SURGE_LOOKUP,
    QUANTILES,
    STORAGE_OPTIONS,
    start_dask_cluster,
)

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [3]:
# When running on larger/scalable dask cluster, may wish to specify number of workers
# Default is LocalCluster which will use the number of CPUs available on local machine
# N_WORKERS_MIN = 7
# N_WORKERS_MAX = 700
SEG_CHUNKSIZE = 5

PARAMS = pd.read_json(PATH_PARAMS)["values"]

In [4]:
DMF_I = getattr(damage_funcs, PARAMS.dmf + "_i")
DDF_I = getattr(damage_funcs, PARAMS.ddf + "_i")

In [5]:
client = start_dask_cluster(
    # n_workers_min=N_WORKERS_MIN,
    # n_workers_max=N_WORKERS_MAX,
)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/daskhub-dev.e0f56d1ca0ae4a42b9352ad6b7204454/status,


## Run surge damage calculations for each combo

In [8]:
futs = {}
for kind, sliiders in [("seg_adm", PATH_SLIIDERS), ("seg", PATH_SLIIDERS_SEG)]:
    futs[kind] = create_surge_lookup(
        sliiders,
        [PATH_SLR_AR5_QUANTILES, PATH_SLR_AR6, PATH_SLR_SWEET],
        PATHS_SURGE_LOOKUP[kind],
        kind,
        PARAMS.at_start,
        PARAMS.n_interp_pts_lslr,
        PARAMS.n_interp_pts_rhdiff,
        DDF_I,
        DMF_I,
        quantiles=QUANTILES,
        start_year=PARAMS.model_start,
        slr_0_years=PARAMS.slr_0_year,
        client=client,
        # client_kwargs={"batch_size": N_WORKERS_MAX},
        force_overwrite=True,
        seg_chunksize=SEG_CHUNKSIZE,
        mc_dim="quantile",
        storage_options=STORAGE_OPTIONS,
    )

## Close

In [9]:
# ensure completion and close cluster
all_futs = futs["seg"] + futs["seg_adm"]
dd.wait(all_futs)
assert [f.status == "finished" for f in all_futs]
finished = True

In [10]:
client.cluster.close(), client.close()

(None, None)